In [6]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.model_selection import train_test_split
from torchvision.models import resnet18, ResNet18_Weights

In [7]:
# # Importing training data

# train_data1 = np.load('../DL-Project/data0.npy')
# train_lab1 = np.load('../DL-Project/lab0.npy')

# train_data2 = np.load('../DL-Project/data1.npy')
# train_lab2 = np.load('../DL-Project/lab1.npy')

# train_data3 = np.load('../DL-Project/data2.npy')
# train_lab3 = np.load('../DL-Project/lab2.npy')

In [8]:
# # Loading and preprocessing - normalizing the image and adding new dimension for convolutions

# images = np.concatenate((train_data1, train_data2, train_data3)) / 255
# labels = np.concatenate((train_lab1, train_lab2, train_lab3))

# images = images[:, np.newaxis, :, :]

In [9]:
test_data = np.load('../DL-Project/data9.npy')
labels = np.load('../DL-Project/label9.npy')

images = test_data / 255
images = images[:, np.newaxis, :, :]

In [10]:
# Converting to pytorch tensors

test_images = torch.tensor(images, dtype=torch.float32)
test_labels = torch.tensor(labels, dtype=torch.long)

In [11]:
# Creating the Dataset from the tensors

class ImageDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

test_dataset = ImageDataset(test_images, test_labels)

# Creating DataLoader
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [12]:
# Choosing GPU for inference

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [13]:
# Load Pretrained ResNet-18

model = resnet18(weights=ResNet18_Weights.DEFAULT)

In [14]:
# Modify First Convolution Layer (ResNet expects 3 channels, but we have 1)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Modify Fully Connected Layer for 41 classes (sums from 0-40)
num_ftrs = model.fc.in_features  # Get input features of last layer
model.fc = nn.Linear(num_ftrs, 41)  # Output layer with 41 classes

model.load_state_dict(torch.load("model.pth"))
print(model)  # Check the model architecture

/tmp/ipykernel_20449/3524786388.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [15]:
# Function to test the Model on the whole dataset

model = model.to(device)

def evaluate_model(model, data_loader):
    model.eval()  # Set to evaluation mode
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predicted_classes = torch.argmax(outputs, dim=1)  # Get predicted class

            correct += (predicted_classes == labels).sum().item()
            total += labels.size(0)

    return 100 * correct / total  # Return accuracy percentage

In [16]:
accuracy = evaluate_model(model, test_loader)

In [17]:
print(accuracy)

93.1


In [18]:
# Function to evaluate L1 loss of model, to compare with baseline CNN 

criterion = nn.L1Loss()

def evaluate_l1(model, test_loader):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            # labels = labels.unsqueeze(1)
            outputs = torch.argmax(model(images), dim=1)
            loss = criterion(outputs, labels.to(torch.float32))
            test_loss += loss.item()
    print(f"Test Loss: {test_loss/len(test_loader):.4f}")

In [19]:
# L1 loss of the model on test set

evaluate_l1(model, test_loader)

Test Loss: 0.1758
